In [1]:
import pandas as pd
import sqlite3


In [3]:
# Load the dataset into a Pandas DataFrame
file_path = '/content/Gift_Store_Sales_2024.csv'  # Update this path
sales_data = pd.read_csv(file_path)

In [4]:
# Normalize column names for SQLite compatibility
sales_data.columns = sales_data.columns.str.replace(" ", "_")


In [5]:
# Create SQLite database in memory
conn = sqlite3.connect(":memory:")


In [6]:
# Load the DataFrame into an SQLite table
sales_data.to_sql("sales_data", conn, index=False, if_exists="replace")


1000

In [7]:
# 1. Total units sold for each item
query_1 = """
SELECT Category AS item_name, SUM(Quantity_Sold) AS total_units_sold
FROM sales_data
GROUP BY item_name
ORDER BY total_units_sold DESC;
"""
result_1 = pd.read_sql_query(query_1, conn)

In [16]:
print("1. Total units sold for each item")
print(result_1)

1. Total units sold for each item
          item_name  total_units_sold
0     Starry Lights               391
1   Christmas Cards               386
2       Candy Stick               382
3  Tree Decor Balls               378
4              Toys               374
5     Gift Wrappers               368
6    Christmas Tree               349
7   Advent Calendar               346


In [8]:
# 2. Revenue generated by each item
query_2 = """
SELECT Category AS item_name, SUM(Quantity_Sold * Price_per_Unit) AS revenue
FROM sales_data
GROUP BY item_name
ORDER BY revenue DESC;
"""
result_2 = pd.read_sql_query(query_2, conn)

In [17]:
print("\n2. Revenue generated by each item")
print(result_2)


2. Revenue generated by each item
          item_name  revenue
0    Christmas Tree    52350
1   Advent Calendar    17300
2  Tree Decor Balls    11340
3       Candy Stick     5730
4   Christmas Cards     3860
5     Starry Lights     3128
6              Toys     1870
7     Gift Wrappers      736


In [9]:
# 3. Aggregate sales by product
query_3 = """
SELECT Category AS item_name,
       SUM(Quantity_Sold) AS total_units_sold,
       SUM(Quantity_Sold * Price_per_Unit) AS total_revenue
FROM sales_data
GROUP BY item_name
ORDER BY total_revenue DESC;
"""
result_3 = pd.read_sql_query(query_3, conn)

In [18]:
print("\n3. Aggregate sales by product")
print(result_3)


3. Aggregate sales by product
          item_name  total_units_sold  total_revenue
0    Christmas Tree               349          52350
1   Advent Calendar               346          17300
2  Tree Decor Balls               378          11340
3       Candy Stick               382           5730
4   Christmas Cards               386           3860
5     Starry Lights               391           3128
6              Toys               374           1870
7     Gift Wrappers               368            736


In [10]:
# 4. Rank items by units sold or revenue
query_4 = """
SELECT Category AS item_name,
       SUM(Quantity_Sold) AS total_units_sold,
       RANK() OVER (ORDER BY SUM(Quantity_Sold) DESC) AS rank_by_units
FROM sales_data
GROUP BY item_name;
"""
result_4 = pd.read_sql_query(query_4, conn)


In [19]:
print("\n4. Rank items by units sold")
print(result_4)


4. Rank items by units sold
          item_name  total_units_sold  rank_by_units
0     Starry Lights               391              1
1   Christmas Cards               386              2
2       Candy Stick               382              3
3  Tree Decor Balls               378              4
4              Toys               374              5
5     Gift Wrappers               368              6
6    Christmas Tree               349              7
7   Advent Calendar               346              8


In [11]:
query_4b = """
SELECT Category AS item_name,
       SUM(Quantity_Sold * Price_per_Unit) AS total_revenue,
       RANK() OVER (ORDER BY SUM(Quantity_Sold * Price_per_Unit) DESC) AS rank_by_revenue
FROM sales_data
GROUP BY item_name;
"""
result_4b = pd.read_sql_query(query_4b, conn)


In [20]:
print("\n4b. Rank items by revenue")
print(result_4b)


4b. Rank items by revenue
          item_name  total_revenue  rank_by_revenue
0    Christmas Tree          52350                1
1   Advent Calendar          17300                2
2  Tree Decor Balls          11340                3
3       Candy Stick           5730                4
4   Christmas Cards           3860                5
5     Starry Lights           3128                6
6              Toys           1870                7
7     Gift Wrappers            736                8


In [12]:
# 5. Total sales and number of transactions by date
query_5 = """
SELECT Date AS sale_date,
       SUM(Quantity_Sold * Price_per_Unit) AS total_sales,
       COUNT(*) AS transaction_count
FROM sales_data
GROUP BY sale_date
ORDER BY sale_date;
"""
result_5 = pd.read_sql_query(query_5, conn)

In [21]:
print("\n5. Total sales and number of transactions by date")
print(result_5)


5. Total sales and number of transactions by date
     sale_date  total_sales  transaction_count
0   2024-12-01         6832                 44
1   2024-12-02         4461                 37
2   2024-12-03         3868                 53
3   2024-12-04         3481                 32
4   2024-12-05         4460                 48
5   2024-12-06         5058                 43
6   2024-12-07         4294                 34
7   2024-12-08         2593                 29
8   2024-12-09         3446                 44
9   2024-12-10         2740                 39
10  2024-12-11         3937                 45
11  2024-12-12         4674                 52
12  2024-12-13         2918                 34
13  2024-12-14         2904                 33
14  2024-12-15         2702                 44
15  2024-12-16         3011                 33
16  2024-12-17         3337                 35
17  2024-12-18         6801                 51
18  2024-12-19         3021                 33
19  2024-

In [13]:
# 6. Identify trends: Sales by day of the week
query_6 = """
SELECT strftime('%w', Date) AS day_of_week,
       SUM(Quantity_Sold * Price_per_Unit) AS total_sales
FROM sales_data
GROUP BY day_of_week
ORDER BY day_of_week;
"""
result_6 = pd.read_sql_query(query_6, conn)

In [22]:
print("\n6. Identify trends: Sales by day of the week")
print(result_6)


6. Identify trends: Sales by day of the week
  day_of_week  total_sales
0           0        14692
1           1        14537
2           2        13299
3           3        18017
4           4        12155
5           5        12609
6           6        11005


In [14]:
# 7. Total revenue for the Christmas period (Dec 1–Dec 31)
query_7 = """
SELECT SUM(Quantity_Sold * Price_per_Unit) AS christmas_revenue
FROM sales_data
WHERE Date BETWEEN '2024-12-01' AND '2024-12-31';
"""
result_7 = pd.read_sql_query(query_7, conn)

In [23]:
print("\n7. Total revenue for the Christmas period")
print(result_7)


7. Total revenue for the Christmas period
   christmas_revenue
0              96314


In [15]:
# 8. Percent change from last year (assuming there are multiple years)
query_8 = """
WITH revenue_by_year AS (
    SELECT strftime('%Y', Date) AS year,
           SUM(Quantity_Sold * Price_per_Unit) AS total_revenue
    FROM sales_data
    GROUP BY year
)
SELECT year,
       total_revenue,
       (total_revenue - LAG(total_revenue) OVER (ORDER BY year)) * 100.0 / LAG(total_revenue) OVER (ORDER BY year) AS percent_change
FROM revenue_by_year;
"""
result_8 = pd.read_sql_query(query_8, conn)

In [24]:
print("\n8. Percent change from last year")
print(result_8)


8. Percent change from last year
   year  total_revenue percent_change
0  2024          96314           None


In [30]:
# Total units sold for each item
query_1 = """
SELECT Category AS item_name, SUM(Quantity_Sold) AS total_units_sold
FROM sales_data
GROUP BY item_name
ORDER BY total_units_sold DESC;
"""
total_units_sold = pd.read_sql_query(query_1, conn)

# Revenue generated by each item
query_2 = """
SELECT Category AS item_name, SUM(Quantity_Sold * Price_per_Unit) AS total_revenue
FROM sales_data
GROUP BY item_name
ORDER BY total_revenue DESC;
"""
total_revenue = pd.read_sql_query(query_2, conn)

# Aggregate sales by product
query_3 = """
SELECT Category AS item_name,
       SUM(Quantity_Sold) AS total_units_sold,
       SUM(Quantity_Sold * Price_per_Unit) AS total_revenue
FROM sales_data
GROUP BY item_name
ORDER BY total_revenue DESC;
"""
aggregate_sales = pd.read_sql_query(query_3, conn)

# Total sales and number of transactions by date
query_4 = """
SELECT Date AS sale_date,
       SUM(Quantity_Sold * Price_per_Unit) AS total_sales,
       COUNT(*) AS transaction_count
FROM sales_data
GROUP BY sale_date
ORDER BY sale_date;
"""
sales_by_date = pd.read_sql_query(query_4, conn)

# Revenue trends by day of the week
query_5 = """
SELECT strftime('%w', Date) AS day_of_week,
       SUM(Quantity_Sold * Price_per_Unit) AS total_sales
FROM sales_data
GROUP BY day_of_week
ORDER BY day_of_week;
"""
trends_by_day = pd.read_sql_query(query_5, conn)

# Total revenue for Christmas period (Dec 1–Dec 31)
query_6 = """
SELECT SUM(Quantity_Sold * Price_per_Unit) AS christmas_revenue
FROM sales_data
WHERE Date BETWEEN '2024-12-01' AND '2024-12-31';
"""
christmas_revenue = pd.read_sql_query(query_6, conn)

# Save all metrics to CSV
total_units_sold.to_csv('/content/Gift_Store_Sales_2024 copy.csv', index=False)



In [31]:
total_revenue.to_csv('/content/Gift_Store_Sales_2024 copy.csv', index=False)

In [32]:
aggregate_sales.to_csv('/content/Gift_Store_Sales_2024 copy.csv', index=False)

In [33]:
sales_by_date.to_csv('/content/Gift_Store_Sales_2024 copy.csv', index=False)

In [34]:
trends_by_day.to_csv('/content/Gift_Store_Sales_2024 copy.csv', index=False)

In [35]:
christmas_revenue.to_csv('/content/Gift_Store_Sales_2024 copy.csv', index=False)

In [28]:
print("All metrics have been saved as CSV files.")

All metrics have been saved as CSV files.
